# Kyle Model Example

This notebook demonstrates the Kyle (1985) model of insider trading with an informed trader, market maker, and noise traders.

## Section 1: Import Required Libraries

In [1]:
import sys
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add the src directory to the path to import our models
src_path = Path('.').resolve() / 'src' if (Path('.').resolve() / 'src').exists() else Path('.').resolve().parent / 'src'
sys.path.insert(0, str(src_path))

from lob_modeling.models.kyle import KyleModel


## Section 2: Kyle Model Overview

The Kyle model captures the interaction between:
- **Informed trader**: Has private information about the true security value
- **Market maker**: Sets prices based on the order flow (informed + noise orders)
- **Noise traders**: Submit random orders, providing liquidity

Key parameters:
- `V_0`: Initial security value
- `V_N`: True security value at end of period
- `SIGMA_G`: Uncertainty about the true value
- `SIGMA`: Noise trader order flow volatility
- `N`: Number of time periods

## Section 3: Initialize the Model

In [2]:
# Initialize the Kyle model with default parameters
model = KyleModel(
    V_0=5.0,           # Initial security value
    V_N=5.0,           # We'll update this randomly
    SIGMA_G=0.4,       # Initial uncertainty about true value
    SIGMA_T=0.2,       # Target true variance
    SIGMA=2.0,         # Noise trader order flow volatility
    ERR=0.05,          # Convergence error tolerance
    N=50,              # Number of time periods
    MAX_ITER=100       # Max iterations for convergence
)

print("Kyle Model initialized with the following parameters:")
print(f"  Initial security value (V_0): {model.V_0}")
print(f"  True security value (V_N): {model.V_N[0]:.4f}")
print(f"  Initial uncertainty (SIGMA_G): {model.SIGMA_G}")
print(f"  Target variance (SIGMA_T): {model.SIGMA_T}")
print(f"  Noise trader volatility (SIGMA): {model.SIGMA}")
print(f"  Number of periods (N): {model.N}")

Kyle Model initialized with the following parameters:
  Initial security value (V_0): 5.0
  True security value (V_N): 4.7642
  Initial uncertainty (SIGMA_G): 0.4
  Target variance (SIGMA_T): 0.2
  Noise trader volatility (SIGMA): 2.0
  Number of periods (N): 50


## Section 4: One-Period Model

In the single-period version, the informed trader observes the true value and submits an optimal order. The market maker observes total order flow and sets the price.

In [3]:
# Run the one-period Kyle model
print("="*60)
print("ONE-PERIOD KYLE MODEL")
print("="*60)
one_period_results = model.one_period_price()
print(f"\nMM Price: ${one_period_results['MM Price']:.4f}")
print(f"Informed Trader Expected Profit: ${one_period_results['Informed Profit']:.4f}")

ONE-PERIOD KYLE MODEL
Market Maker Price: 10.106242157322221
Informed Trader Expected Profit: 0.0879217341606021

MM Price: $10.1062
Informed Trader Expected Profit: $0.0879


## Section 5: Multi-Period Model

In the multi-period version, the model iteratively solves for optimal pricing schedules and order sizes across multiple trading rounds. The algorithm converges on the volatility of the informed trader's information.

In [4]:
# Initialize a fresh model for multi-period analysis
model_mp = KyleModel(
    V_0=5.0,
    V_N=5.0,
    SIGMA_G=0.4,
    SIGMA_T=0.2,
    SIGMA=2.0,
    ERR=0.05,
    N=50,
    MAX_ITER=100
)

print("\n" + "="*60)
print("MULTI-PERIOD KYLE MODEL")
print("="*60)
print(f"\nTarget true volatility (SIGMA_T): {model_mp.SIGMA_T}")
print(f"Initial guess for SIGMA[0]: {model_mp.SIGMA}")
print("\nSolving multi-period equilibrium...")

# Run the multi-period model without plotting during convergence
model_mp.multiperiod_price(plot=True)


MULTI-PERIOD KYLE MODEL

Target true volatility (SIGMA_T): 0.2
Initial guess for SIGMA[0]: 2.0

Solving multi-period equilibrium...


## Section 6: Model Interpretation

The Kyle model reveals several important insights about market microstructure:

1. **Information Advantage**: The informed trader's profit increases with the information advantage (difference between V_N and V_0)

2. **Order Flow as Signal**: The market maker learns from order flow. Higher order flow reveals more information.

3. **Volatility Trading-off**: There's a trade-off between:
   - Larger informed order sizes (more profitable for insider)
   - Larger spreads (market maker compensates for adverse selection risk)

4. **Time Decay**: In multi-period models, the informed trader gradually reveals information through dynamic trading.

## Section 7: Practical Applications

The Kyle model can be extended to study:

- **Market Efficiency**: How quickly does private information get incorporated into prices?
- **Bid-Ask Spreads**: How do spreads vary with information asymmetry?
- **Trading Volume**: Why do informed traders sometimes submit large orders despite revealing their private information?
- **Regulation**: How do position limits or disclosure requirements affect information revelation?

The model provides a foundation for understanding how informed traders strategically submit orders to maximize profit while managing information leakage.